In [21]:
import numpy as np
import pymysql
import os
import cv2
from collections import namedtuple

User = namedtuple('User', ['table', 'load_l2_dir', 'load_raw_dir'])
self = User(table="face_data",load_l2_dir="./save/l2.npy" ,load_raw_dir="./save/raw.npy" )
input_dir = "./pic3"
replace_flag =True
db = pymysql.connect(host="localhost", user="root", password="qiqi39", port=3306, charset="utf8" )
cursor = db.cursor()
# cursor.execute("DROP DATABASE IF EXISTS face_db")
cursor.execute("CREATE DATABASE IF NOT EXISTS face_db")
cursor.execute("USE face_db;")
cursor.execute("ALTER database face_db character SET gbk;")
cursor.execute("CREATE TABLE IF NOT EXISTS face_data(FaceName VARCHAR(20), EmbdNums INT, ColumnNum INT);")



input_lists = os.listdir(input_dir)
cursor.execute("alter table %s add Crement int;"%(self.table))
cursor.execute("alter table %s change crement crement int not null auto_increment primary key;"%(self.table))
cursor.execute("SELECT Crement-1 FROM %s WHERE Crement-1 NOT IN (SELECT ColumnNum FROM %s) ;"%(self.table,self.table))
ColumnNum_data_loss = cursor.fetchall()
print(ColumnNum_data_loss)
if not os.path.exists(self.load_l2_dir):
    label = list(range(len(input_lists)))
    embds_db = np.zeros((len(input_lists)+1, 1, 512))
    embds = np.zeros((len(input_lists)+1, 1, 512))
else:
    embds_db = np.load(self.load_l2_dir) 
    embds = np.load(self.load_raw_dir)
    row,_,_ = embds_db.shape        
    label_data = [_[0] for _ in ColumnNum_data_loss]
    print(label_data)
    expand_label = list(range(row,row + len(input_lists) - len(label_data)))
    label = label_data + expand_label
    
    print(label)
    print(expand_label)
    print(embds_db.shape)    
    embds_db = np.row_stack((embds_db, np.zeros((len(expand_label)+1, 1, 512))))
    embds = np.row_stack((embds, np.zeros((len(expand_label)+1, 1, 512))))
    print(embds_db.shape)
    
nonface_num =0    
idx = 0
kkk=20
embd=np.zeros((1,512))
for person_name in input_lists:

    _subdir = os.path.join(input_dir, person_name)

    if not os.path.isdir(_subdir):
        continue

    num = 0.0
    embd_tp_save = np.zeros((1,512))
    for img_name in os.listdir(_subdir):

        image_path = os.path.join(_subdir, img_name)
        
        embd+=1

        num += 1
        embd_tp_save += embd

    if(all((embd_tp_save==np.zeros((1,512)))[0])):
        print(person_name+" has no face to detect")
        nonface_num+=1
        continue
    else:
        embd_tp_save /= num   
        
    cursor.execute("select count(*) from %s where FaceName ='%s';"%(self.table, person_name))
    name_exists = cursor.fetchall()[0][0]

    if name_exists:
        
        nonface_num+=1
        cursor.execute("select EmbdNums,ColumnNum from %s where FaceName ='%s';"%(self.table, person_name))
        EmbdNums,ColumnNum = cursor.fetchall()[0]         
        if not replace_flag:
 
            embd_all = (embds[ColumnNum,:] * EmbdNums) + (embd_tp_save * num)
            embds[ColumnNum,:] = embd_all/(EmbdNums + num)
            embds_db[ColumnNum,:] = embd_all/np.linalg.norm(embd_all, axis=1, keepdims=True)
        else:
            embds_db[ColumnNum,:] = embd_tp_save/np.linalg.norm(embd_tp_save, axis=1, keepdims=True)
            embds[ColumnNum,:] = embd_tp_save
            EmbdNums = 0

        try:
            cursor.execute("UPDATE %s SET EmbdNums = %d WHERE FaceName = '%s';"%(self.table, EmbdNums + num, person_name))
            db.commit()
        except:
            db.rollback()
            raise Exception('''mysql "UPDATE" action error in FaceName %d"'''%(person_name))

    else:    
        embds[label[idx],:] = embd_tp_save
        embds_db[label[idx],:] = embd_tp_save/np.linalg.norm(embd_tp_save, axis=1, keepdims=True)
        # num represents numbers of embd,label represents the column number

        try:
            cursor.execute("INSERT INTO face_data(FaceName,EmbdNums, ColumnNum) VALUES ('%s', %.4f, %d)"%(person_name, num, label[idx]))
            db.commit()
        except:
            db.rollback()
            raise Exception('''mysql "INSERT" action error in label %d"'''%(label[idx]))

        idx += 1

np.save(self.load_l2_dir, embds_db[:-1-nonface_num,])  
np.save(self.load_raw_dir, embds[:-1-nonface_num,]) 

cursor.execute("alter table %s drop column Crement;"%(self.table))

D:\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1007, "Can't create database 'face_db'; database exists")
  result = self._query(query)
D:\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1050, "Table 'face_data' already exists")
  result = self._query(query)


()
[]
[5]
[5]
(5, 1, 512)
(7, 1, 512)


5

In [11]:
cursor.execute("alter table %s drop column Crement;"%(self.table))

4

In [4]:
cursor.execute("delete from %s where FaceName='%s';"%(self.table, "c"))
db.commit()

In [22]:
cursor.execute("SELECT * FROM face_data;")

cursor.fetchall()

(('a', 5, 0), ('b', 1, 1), ('d', 6, 3), ('fw', 1, 2), ('e', 1, 4))

In [23]:
np.load(self.load_raw_dir)

array([[[ 1.4,  1.4,  1.4, ...,  1.4,  1.4,  1.4]],

       [[ 3. ,  3. ,  3. , ...,  3. ,  3. ,  3. ]],

       [[ 1. ,  1. ,  1. , ...,  1. ,  1. ,  1. ]],

       [[10.5, 10.5, 10.5, ..., 10.5, 10.5, 10.5]],

       [[ 3. ,  3. ,  3. , ...,  3. ,  3. ,  3. ]]])

In [24]:
try:
    a=1/0
    
except:
    print(1)

1


In [19]:
np.load(self.load_l2_dir)

array([[[0.04419417, 0.04419417, 0.04419417, ..., 0.04419417,
         0.04419417, 0.04419417]],

       [[0.04419417, 0.04419417, 0.04419417, ..., 0.04419417,
         0.04419417, 0.04419417]],

       [[0.04419417, 0.04419417, 0.04419417, ..., 0.04419417,
         0.04419417, 0.04419417]],

       [[0.04419417, 0.04419417, 0.04419417, ..., 0.04419417,
         0.04419417, 0.04419417]],

       [[0.04419417, 0.04419417, 0.04419417, ..., 0.04419417,
         0.04419417, 0.04419417]]])

In [1]:
import pymysql
db = pymysql.connect(host="localhost", user="root", password="qiqi39", port=3306, charset="utf8" )
cursor = db.cursor()
cursor.execute("DROP DATABASE IF EXISTS face_db")
cursor.execute("CREATE DATABASE IF NOT EXISTS face_db")
cursor.execute("USE face_db;")
cursor.execute("ALTER database face_db character SET gbk;")
cursor.execute("CREATE TABLE IF NOT EXISTS face_data(FaceName VARCHAR(20), EmbdNums INT, ColumnNum INT);")

0

In [4]:
cursor.execute("desc face_data;")
cursor.fetchall()

(('FaceName', 'varchar(20)', 'YES', '', None, ''),
 ('EmbdNums', 'int(11)', 'YES', '', None, ''),
 ('ColumnNum', 'int(11)', 'YES', '', None, ''))

In [2]:

# DROP PROCEDURE IF EXISTS schema_change;  
# DELIMITER $ 
# CREATE PROCEDURE schema_change() 
# BEGIN DECLARE  CurrentDatabase VARCHAR(100);
# SELECT DATABASE() INTO CurrentDatabase;
# IF EXISTS (SELECT * FROM information_schema.columns 
# WHERE table_schema=CurrentDatabase AND table_name = 'face_data' AND column_name = 'ColumnNum') 
# THEN  ALTER TABLE face_data DROP COLUMN ColumnNum;
# END IF;  
# END $  
# DELIMITER ;  
# CALL schema_change();

# cursor.execute(sql)
# cursor.execute("desc face_data;")
# cursor.fetchall()

In [2]:
cursor.execute("SELECT * FROM face_data;")
cursor.fetchall()

()

In [34]:
cursor.execute("select count(*) from information_schema.columns where table_name = 'face_data' and column_name = 'FaceName';")
cursor.fetchall()

((1,),)

In [1]:
import cv2
img = cv2.imread("./mage-2019-3.png")

In [1]:
import cv2
img = cv2.imread("./mage-2019-3.png")
cv2.rectangle(img, (30,30), (80, 80), (255, 255, 255), thickness=2)
cv2.putText(img, "haha", (30,30), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0),
           thickness=2)
cv2.imshow('head', img)
cv2.waitKey(0)

-1

In [ ]:

cv.rectangle(img, (x1, y1), (x2, y2), (255, 255, 255), thickness=2)
cv.putText(img, objectname, (x1, y1), cv.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0),
           thickness=2)
cv.imshow('head', img)
